In [1]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
# from rnn_tools.model_saver import model_saver, model_saver_wandb
# import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data
import rnn_tools.training_testing_gru_extra_data_embedding as training_testing_gru_extra_data_embedding

c:\Users\Nick\Documents\GitHub\grvmodel\python


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")


os.environ["WANDB_NOTEBOOK_NAME"] = "GRU - v8.0.1 Topaz.ipynb"

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [4]:
torch.__version__

'2.1.1'

In [5]:
data_file = './data/topaz_data_w_bsp_new5_nz.fth'
df = pd.read_feather(data_file)

In [6]:
df.BSP

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
391881   NaN
391882   NaN
391883   NaN
391884   NaN
391885   NaN
Name: BSP, Length: 391886, dtype: float64

In [7]:
os.getcwd()
import rnn_tools.raceDB
importlib.reload(rnn_tools.raceDB)
import rnn_tools.model_saver
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")                                                                                                                                                                                                                                                                                                                
date = datetime.datetime.strptime("2018-01-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ","SA","WA"]
# states = ["VIC"]
states = ["NSW"]
# states = ["NZ"]
# states = ['VIC']
extra_tracks = ["Wentworth Park", "Wagga", "Temora"]
only_tracks = ["Bendigo", "Geelong", "Traralgon", "Ballarat", "Shepparton", "Warragul","Healesville"]
data_file = './data/gru_inputs_kitchen_sink_new_not_simple.fth'
# data_file = './data/topaz_data_w_bsp_new4_small.fth'
# data_file = './data/topaz_data_w_bsp_new4.fth'
data_file = './data/topaz_data_w_bsp_new7.fth'
   
raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,track_filter=extra_tracks,only_tracks=only_tracks,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='boosted_sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,track_filter=extra_tracks,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
raceDB.create_new_weights_v2()

c:\Users\Nick\Documents\GitHub\grvmodel\python
(2518990, 196)
80
Latest date = 2024-05-19 00:00:00
size after state filter (2518990, 197)
(859808, 198)
Index(['index', 'meetingDate', 'state', 'track_name', 'dist', 'raceId',
       'race_grade', 'race_num', 'box', 'rugNumber',
       ...
       'prev_BSP_price', 'prev_start_price', 'stats_topaz', 'dogid', 'raceid',
       'stats_cols', 'track_hash', 'stats', 'race_time', 'stats_cuda'],
      dtype='object', length=199)
state  track_name       
NSW    Armidale               309
       Bathurst             20483
       Broken Hill           6203
       Bulli                22768
       Canberra              1482
       Casino               29844
       Coonabarabran         1296
       Coonamble             4743
       Cowra                 1438
       Dapto                24988
       Dubbo                24911
       Gosford              24419
       Goulburn             30932
       Grafton              28826
       Gunnedah           

100%|██████████| 25/25 [00:00<00:00, 25007.77it/s]


race_ids=['284942891', '285107241', '285107243', '285107242', '285107244', '285107245', '285107246', '285107248', '285107247', '285107250', '285107249', '284942890', '284942889', '284942886', '284942888', '284942887', '284942885', '284942884', '284942892', '284942893', '284840745', '284840744', '284840740', '284840743', '284840742', '284840741', '284840737', '284840738', '284840739', '284840736', '284840735', '284942901', '284942913', '285107230', '284942917', '284942915', '284942910', '284942907', '284942896', '284942905', '284942903', '285107229', '284942899', '285107262', '284942935', '284840798', '284942936', '284942937', '284942938', '284942939', '284942940', '284942946', '284942943', '284942941', '284942942', '284942944', '284942945', '285107266', '285107267', '285107264', '285107263', '285107265', '285107268', '285107269', '285107270', '285107271', '285107272', '284942960', '284942951', '284942956', '284942964', '284942961', '284942950', '284942949', '284942959', '284942962', '2

In [8]:
feature_cols = ['dam_finishingPlaceMovement_max_28D',
 'dam_finishingPlaceMovement_max_365D',
 'dam_finishingPlaceMovement_max_91D',
 'dam_finishingPlaceMovement_mean_28D',
 'dam_finishingPlaceMovement_mean_365D',
 'dam_finishingPlaceMovement_mean_91D',
 'dam_finishingPlaceMovement_median_28D',
 'dam_finishingPlaceMovement_median_365D',
 'dam_finishingPlaceMovement_median_91D',
 'dam_finishingPlaceMovement_min_28D',
 'dam_finishingPlaceMovement_min_365D',
 'dam_finishingPlaceMovement_min_91D',
 'dam_finishingPlaceMovement_std_28D',
 'dam_finishingPlaceMovement_std_365D',
 'dam_finishingPlaceMovement_std_91D',
 'dam_marginLog_max_28D',
 'dam_marginLog_max_365D',
 'dam_marginLog_max_91D',
 'dam_marginLog_mean_28D',
 'dam_marginLog_mean_365D',
 'dam_marginLog_mean_91D',
 'dam_marginLog_median_28D',
 'dam_marginLog_median_365D',
 'dam_marginLog_median_91D',
 'dam_marginLog_min_28D',
 'dam_marginLog_min_365D',
 'dam_marginLog_min_91D',
 'dam_marginLog_std_28D',
 'dam_marginLog_std_365D',
 'dam_marginLog_std_91D',
 'dam_placeLog_max_28D',
 'dam_placeLog_max_365D',
 'dam_placeLog_max_91D',
 'dam_placeLog_mean_28D',
 'dam_placeLog_mean_365D',
 'dam_placeLog_mean_91D',
 'dam_placeLog_median_28D',
 'dam_placeLog_median_365D',
 'dam_placeLog_median_91D',
 'dam_placeLog_min_28D',
 'dam_placeLog_min_365D',
 'dam_placeLog_min_91D',
 'dam_placeLog_std_28D',
 'dam_placeLog_std_365D',
 'dam_placeLog_std_91D',
 'dam_prizemoneyLog_max_28D',
 'dam_prizemoneyLog_max_365D',
 'dam_prizemoneyLog_max_91D',
 'dam_prizemoneyLog_mean_28D',
 'dam_prizemoneyLog_mean_365D',
 'dam_prizemoneyLog_mean_91D',
 'dam_prizemoneyLog_median_28D',
 'dam_prizemoneyLog_median_365D',
 'dam_prizemoneyLog_median_91D',
 'dam_prizemoneyLog_min_28D',
 'dam_prizemoneyLog_min_365D',
 'dam_prizemoneyLog_min_91D',
 'dam_prizemoneyLog_std_28D',
 'dam_prizemoneyLog_std_365D',
 'dam_prizemoneyLog_std_91D',
 'dam_runTimeNorm_max_28D',
 'dam_runTimeNorm_max_365D',
 'dam_runTimeNorm_max_91D',
 'dam_runTimeNorm_mean_28D',
 'dam_runTimeNorm_mean_365D',
 'dam_runTimeNorm_mean_91D',
 'dam_runTimeNorm_median_28D',
 'dam_runTimeNorm_median_365D',
 'dam_runTimeNorm_median_91D',
 'dam_runTimeNorm_min_28D',
 'dam_runTimeNorm_min_365D',
 'dam_runTimeNorm_min_91D',
 'dam_runTimeNorm_std_28D',
 'dam_runTimeNorm_std_365D',
 'dam_runTimeNorm_std_91D',
 'dog_finishingPlaceMovement_max_28D',
 'dog_finishingPlaceMovement_max_365D',
 'dog_finishingPlaceMovement_max_91D',
 'dog_finishingPlaceMovement_mean_28D',
 'dog_finishingPlaceMovement_mean_365D',
 'dog_finishingPlaceMovement_mean_91D',
 'dog_finishingPlaceMovement_median_28D',
 'dog_finishingPlaceMovement_median_365D',
 'dog_finishingPlaceMovement_median_91D',
 'dog_finishingPlaceMovement_min_28D',
 'dog_finishingPlaceMovement_min_365D',
 'dog_finishingPlaceMovement_min_91D',
 'dog_finishingPlaceMovement_std_28D',
 'dog_finishingPlaceMovement_std_365D',
 'dog_finishingPlaceMovement_std_91D',
 'dog_marginLog_max_28D',
 'dog_marginLog_max_365D',
 'dog_marginLog_max_91D',
 'dog_marginLog_mean_28D',
 'dog_marginLog_mean_365D',
 'dog_marginLog_mean_91D',
 'dog_marginLog_median_28D',
 'dog_marginLog_median_365D',
 'dog_marginLog_median_91D',
 'dog_marginLog_min_28D',
 'dog_marginLog_min_365D',
 'dog_marginLog_min_91D',
 'dog_marginLog_std_28D',
 'dog_marginLog_std_365D',
 'dog_marginLog_std_91D',
 'dog_placeLog_max_28D',
 'dog_placeLog_max_365D',
 'dog_placeLog_max_91D',
 'dog_placeLog_mean_28D',
 'dog_placeLog_mean_365D',
 'dog_placeLog_mean_91D',
 'dog_placeLog_median_28D',
 'dog_placeLog_median_365D',
 'dog_placeLog_median_91D',
 'dog_placeLog_min_28D',
 'dog_placeLog_min_365D',
 'dog_placeLog_min_91D',
 'dog_placeLog_std_28D',
 'dog_placeLog_std_365D',
 'dog_placeLog_std_91D',
 'dog_prizemoneyLog_max_28D',
 'dog_prizemoneyLog_max_365D',
 'dog_prizemoneyLog_max_91D',
 'dog_prizemoneyLog_mean_28D',
 'dog_prizemoneyLog_mean_365D',
 'dog_prizemoneyLog_mean_91D',
 'dog_prizemoneyLog_median_28D',
 'dog_prizemoneyLog_median_365D',
 'dog_prizemoneyLog_median_91D',
 'dog_prizemoneyLog_min_28D',
 'dog_prizemoneyLog_min_365D',
 'dog_prizemoneyLog_min_91D',
 'dog_prizemoneyLog_std_28D',
 'dog_prizemoneyLog_std_365D',
 'dog_prizemoneyLog_std_91D',
 'dog_runTimeNorm_max_28D',
 'dog_runTimeNorm_max_365D',
 'dog_runTimeNorm_max_91D',
 'dog_runTimeNorm_mean_28D',
 'dog_runTimeNorm_mean_365D',
 'dog_runTimeNorm_mean_91D',
 'dog_runTimeNorm_median_28D',
 'dog_runTimeNorm_median_365D',
 'dog_runTimeNorm_median_91D',
 'dog_runTimeNorm_min_28D',
 'dog_runTimeNorm_min_365D',
 'dog_runTimeNorm_min_91D',
 'dog_runTimeNorm_std_28D',
 'dog_runTimeNorm_std_365D',
 'dog_runTimeNorm_std_91D',
 'sire_finishingPlaceMovement_max_28D',
 'sire_finishingPlaceMovement_max_365D',
 'sire_finishingPlaceMovement_max_91D',
 'sire_finishingPlaceMovement_mean_28D',
 'sire_finishingPlaceMovement_mean_365D',
 'sire_finishingPlaceMovement_mean_91D',
 'sire_finishingPlaceMovement_median_28D',
 'sire_finishingPlaceMovement_median_365D',
 'sire_finishingPlaceMovement_median_91D',
 'sire_finishingPlaceMovement_min_28D',
 'sire_finishingPlaceMovement_min_365D',
 'sire_finishingPlaceMovement_min_91D',
 'sire_finishingPlaceMovement_std_28D',
 'sire_finishingPlaceMovement_std_365D',
 'sire_finishingPlaceMovement_std_91D',
 'sire_marginLog_max_28D',
 'sire_marginLog_max_365D',
 'sire_marginLog_max_91D',
 'sire_marginLog_mean_28D',
 'sire_marginLog_mean_365D',
 'sire_marginLog_mean_91D',
 'sire_marginLog_median_28D',
 'sire_marginLog_median_365D',
 'sire_marginLog_median_91D',
 'sire_marginLog_min_28D',
 'sire_marginLog_min_365D',
 'sire_marginLog_min_91D',
 'sire_marginLog_std_28D',
 'sire_marginLog_std_365D',
 'sire_marginLog_std_91D',
 'sire_placeLog_max_28D',
 'sire_placeLog_max_365D',
 'sire_placeLog_max_91D',
 'sire_placeLog_mean_28D',
 'sire_placeLog_mean_365D',
 'sire_placeLog_mean_91D',
 'sire_placeLog_median_28D',
 'sire_placeLog_median_365D',
 'sire_placeLog_median_91D',
 'sire_placeLog_min_28D',
 'sire_placeLog_min_365D',
 'sire_placeLog_min_91D',
 'sire_placeLog_std_28D',
 'sire_placeLog_std_365D',
 'sire_placeLog_std_91D',
 'sire_prizemoneyLog_max_28D',
 'sire_prizemoneyLog_max_365D',
 'sire_prizemoneyLog_max_91D',
 'sire_prizemoneyLog_mean_28D',
 'sire_prizemoneyLog_mean_365D',
 'sire_prizemoneyLog_mean_91D',
 'sire_prizemoneyLog_median_28D',
 'sire_prizemoneyLog_median_365D',
 'sire_prizemoneyLog_median_91D',
 'sire_prizemoneyLog_min_28D',
 'sire_prizemoneyLog_min_365D',
 'sire_prizemoneyLog_min_91D',
 'sire_prizemoneyLog_std_28D',
 'sire_prizemoneyLog_std_365D',
 'sire_prizemoneyLog_std_91D',
 'sire_runTimeNorm_max_28D',
 'sire_runTimeNorm_max_365D',
 'sire_runTimeNorm_max_91D',
 'sire_runTimeNorm_mean_28D',
 'sire_runTimeNorm_mean_365D',
 'sire_runTimeNorm_mean_91D',
 'sire_runTimeNorm_median_28D',
 'sire_runTimeNorm_median_365D',
 'sire_runTimeNorm_median_91D',
 'sire_runTimeNorm_min_28D',
 'sire_runTimeNorm_min_365D',
 'sire_runTimeNorm_min_91D',
 'sire_runTimeNorm_std_28D',
 'sire_runTimeNorm_std_365D',
 'sire_runTimeNorm_std_91D',
 'trainer_finishingPlaceMovement_max_28D',
 'trainer_finishingPlaceMovement_max_365D',
 'trainer_finishingPlaceMovement_max_91D',
 'trainer_finishingPlaceMovement_mean_28D',
 'trainer_finishingPlaceMovement_mean_365D',
 'trainer_finishingPlaceMovement_mean_91D',
 'trainer_finishingPlaceMovement_median_28D',
 'trainer_finishingPlaceMovement_median_365D',
 'trainer_finishingPlaceMovement_median_91D',
 'trainer_finishingPlaceMovement_min_28D',
 'trainer_finishingPlaceMovement_min_365D',
 'trainer_finishingPlaceMovement_min_91D',
 'trainer_finishingPlaceMovement_std_28D',
 'trainer_finishingPlaceMovement_std_365D',
 'trainer_finishingPlaceMovement_std_91D',
 'trainer_marginLog_max_28D',
 'trainer_marginLog_max_365D',
 'trainer_marginLog_max_91D',
 'trainer_marginLog_mean_28D',
 'trainer_marginLog_mean_365D',
 'trainer_marginLog_mean_91D',
 'trainer_marginLog_median_28D',
 'trainer_marginLog_median_365D',
 'trainer_marginLog_median_91D',
 'trainer_marginLog_min_28D',
 'trainer_marginLog_min_365D',
 'trainer_marginLog_min_91D',
 'trainer_marginLog_std_28D',
 'trainer_marginLog_std_365D',
 'trainer_marginLog_std_91D',
 'trainer_placeLog_max_28D',
 'trainer_placeLog_max_365D',
 'trainer_placeLog_max_91D',
 'trainer_placeLog_mean_28D',
 'trainer_placeLog_mean_365D',
 'trainer_placeLog_mean_91D',
 'trainer_placeLog_median_28D',
 'trainer_placeLog_median_365D',
 'trainer_placeLog_median_91D',
 'trainer_placeLog_min_28D',
 'trainer_placeLog_min_365D',
 'trainer_placeLog_min_91D',
 'trainer_placeLog_std_28D',
 'trainer_placeLog_std_365D',
 'trainer_placeLog_std_91D',
 'trainer_prizemoneyLog_max_28D',
 'trainer_prizemoneyLog_max_365D',
 'trainer_prizemoneyLog_max_91D',
 'trainer_prizemoneyLog_mean_28D',
 'trainer_prizemoneyLog_mean_365D',
 'trainer_prizemoneyLog_mean_91D',
 'trainer_prizemoneyLog_median_28D',
 'trainer_prizemoneyLog_median_365D',
 'trainer_prizemoneyLog_median_91D',
 'trainer_prizemoneyLog_min_28D',
 'trainer_prizemoneyLog_min_365D',
 'trainer_prizemoneyLog_min_91D',
 'trainer_prizemoneyLog_std_28D',
 'trainer_prizemoneyLog_std_365D',
 'trainer_prizemoneyLog_std_91D',
 'trainer_runTimeNorm_max_28D',
 'trainer_runTimeNorm_max_365D',
 'trainer_runTimeNorm_max_91D',
 'trainer_runTimeNorm_mean_28D',
 'trainer_runTimeNorm_mean_365D',
 'trainer_runTimeNorm_mean_91D',
 'trainer_runTimeNorm_median_28D',
 'trainer_runTimeNorm_median_365D',
 'trainer_runTimeNorm_median_91D',
 'trainer_runTimeNorm_min_28D',
 'trainer_runTimeNorm_min_365D',
 'trainer_runTimeNorm_min_91D',
 'trainer_runTimeNorm_std_28D',
 'trainer_runTimeNorm_std_365D',
 'trainer_runTimeNorm_std_91D',
 'dogAgeScaled',
 'lastFiveWinPercentage',
 'lastFivePlacePercentage',
 'weightInKgScaled',
 'rolling_box_win_percentage']

In [9]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def load_matching_state_dict(model, state_dict):
    model_dict = model.state_dict()

    # Filter out unnecessary keys
    matching_keys = {k: v for k, v in state_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
    excluded_keys = set(state_dict.keys()) - set(matching_keys.keys())

    # Overwrite entries in the existing state dict
    model_dict.update(matching_keys)

    # Load the new state dict
    model.load_state_dict(model_dict)

    print("Loaded keys:")
    for key in matching_keys:
        print(key)

    print("\nExcluded keys:")
    for key in excluded_keys:
        print(key)

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7 Reporting", config=config,save_code=True):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("val_ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      wandb.run.log_code('rnn_tools/')
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      for i in config.stat_list_dict.values():
        print(i)

      reg_stat_mask = []
      for stat,flag in config.stat_list_dict.items():
          stat_flag = flag
          reg_stat_mask = reg_stat_mask+[stat_flag]

      print(reg_stat_mask)
      stat_mask = [1]*6+reg_stat_mask+[0]*80
      # stat_mask = [1]*30
      stat_mask = [1]*27+53*[0]
      data_mask = [0]*26+[1]*20+[1]*20+[1]*20+[1]*20 # Reg, Dist, Box, T_box, T_dist
      data_mask = [1]*6+reg_stat_mask+[0]*20+[0]*20+[0]*20+[0]*20 # Reg, Dist, Box,T_box, T_dist
      data_mask =  [1]*27+53*[1]
      # data_mask =  [1]*30
      data_mask_size = sum(data_mask)
      print(f"{data_mask_size=}")
      print(f"{data_mask=}")
      stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)
      data_mask = torch.tensor(data_mask).type(torch.bool).to(device)
      # stat_mask = torch.ones_like(stat_mask).type(torch.bool).to(device)
      input_size = sum(stat_mask)

      print(stat_mask)
      print(stat_mask.shape)


      if 'batch_days' in config.keys() and not raceDB.batches_setup:
        pass
        raceDB.create_test_split_date(config['training_date_end'],val_date='2023-06-01')
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'], stat_mask=stat_mask,data_mask=data_mask,gen_packed_seq=True)
        raceDB.batches_setup = True
      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          # input_size = raceDB.batches['packed_x'][0].data[0].shape[0]



      print(f"{input_size=}")
      config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_extra_embedding(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=data_mask_size)
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      
      config['model_name'] = model.__class__.__name__
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        print(f"Loading model {prev_model_file}, version {prev_model_version}")
        model_name = prev_model_file
        model_loc = f"models/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())
        load_matching_state_dict(model, model_data['model_state_dict'])
        config['parent model'] = prev_model_file
        model = model.to(device)        
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
      else:
        optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')
      wandb.run.name = f"{' '.join(states)}-{wandb.run.name}"
      model = model.to(device)
      print(model)

      try:
        return training_testing_gru_extra_data_embedding.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      raceDB.create_hidden_states_dict_v2()
      rnn_tools.model_saver.model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")


    return (model,dataset, optimizer)

In [10]:
raceDB.batches_setup = False

In [11]:
stat_list_dict = {'_dist_last__1':1,
    '_box_last__1':1,
    '_speed_avg_1':1,
    '_split_speed_avg_1':0,
    '_split_margin_avg_1':1,
    '_margin_avg_1':1,
    '_margin_time_avg_1':1,
    '_RunHomeTime_1':1,
    '_run_home_speed_1':1,
    '_first_out_avg_1':0,  
    '_pos_out_avg_1':0,
    '_post_change_avg_1':0,
    '_races_1':1,
    '_wins_1':1,
    '_wins_last_1':1,
    '_weight_':1,
    '_min_time_':1,
    '_min_split_time_':1,
    '_last_start_price':0,
    '_last_start_prob':0,
}

wandb_config_static = {
    'hidden_size': 256,
    'stats': raceDB.stats_cols,
    'races': states,
    'datafile': data_file,
    'latest_date': raceDB.latest_date,
    'input_type': 'basic',
    'num_layers': 2,
    'batch_size': 750,
    'dropout': 0.3,
    'epochs': 2500,
    'learning_rate': 0.001,
    'optimizer': 'adamW',
    'f0_layer_size': 128,
    'f1_layer_size': 64,
    'training_date_end': '2022-01-01',
    'notes':'GRU, with basic add on data looped in',
    'batch_days': 450,
    'stat_list_dict': stat_list_dict,
}

In [12]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])
torch.cuda.empty_cache()
raceDB.race_prices_to_prob()
print(raceDB.latest_date)
for race in raceDB.racesDict.values():
    race.loss = torch.tensor(1.0,requires_grad=True,device='cuda:0')  
    race.loss_log = torch.tensor(1.0,requires_grad=True,device='cuda:0')  

for race in raceDB.racesDict.values():
    if race.classes.isnan().sum():
        print(race.classes.sum())
        print(race.raceid)
        race.classes = race.classes.nan_to_num(0,0,0)

2024-05-18 00:00:00


In [13]:
torch.cuda.empty_cache()   

In [14]:
x = torch.rand(10,8)
x.argmax(dim=-1)

tensor([3, 4, 1, 2, 3, 3, 2, 3, 5, 1])

In [15]:
for r in raceDB.racesDict.values():
    try:
        r.start_prob
    except:
        print(r.raceid)
        print(r.raw_prices)

In [16]:
for r in raceDB.racesDict.values():
    print(r.prices)
    # print(all([x==np.nan for x in r.prices]))
    # print(all([(np.isnan(x) or x==0) for x in r.prices]))
    print(r.fin_prob)
    print(r.start_prob)
    break

[8.2, 7.2, 4.9, 9.676760507834915, 3.7, 17.520249278708945, 9.134045437, 42.0]
tensor([0.1183, 0.1353, 0.2006, 0.0997, 0.2670, 0.0533, 0.1058, 0.0200],
       device='cuda:0')
tensor([0.1510, 0.1510, 0.0760, 0.0837, 0.3265, 0.0462, 0.1255, 0.0401],
       device='cuda:0')


In [17]:
for r in raceDB.racesDict.values():
    try:
        r.fin_prob
    except:
        print(r.raceid)
        print(r.raw_prices)
        # r

In [18]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
importlib.reload(training_testing_gru_extra_data_embedding)
importlib.reload(rnn_tools.model_saver)
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='NSW-lilac-sunset-824', prev_model_version = 60)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='VIC SA NSW-ancient-valley-726', prev_model_version = 280) 


wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 256, 'stats': "['dogAgeScaled', 'boxNumber', 'incomingGrade_val', 'weightInKgScaled', 'hasEntryBoxNumberPlus1', 'hasEntryBoxNumberMinus1', 'rolling_box_win_percentage', 'dog_incomingGrade_val_mean_1', 'dog_startPrice_mean_1', 'dog_distance_mean_1', 'dog_boxNumber_mean_1', 'dog_runTimeNorm_mean_1', 'dog_win_runTimeNorm_mean_1', 'dog_delta_win_runTimeNorm_mean_1', 'dog_place_mean_1', 'dog_resultMargin_mean_1', 'dog_finishingPlaceMovement_mean_1', 'dog_StartingPlaceMovement_mean_1', 'dog_split_time_margin_mean_1', 'dog_split_runTimeNorm_mean_1', 'dog_time_1_mean_1', 'dog_run_home_TimeNorm_mean_1', 'dog_performance_to_price_mean_1', 'dog_averageSpeed_mean_1', 'dog_win_mean_1', 'dog_time_1_mean_365D', 'trainer_incomingGrade_val_mean_365D', 'trainer_startPrice_mean_365D', 'trainer_distance_mean_365D', 'trainer_boxNumber_mean_365D', 'trainer_runTimeNorm_mean_365D', 'trainer_win_runTimeNorm_mean_365D', 'trainer_delta_win_runTimeNorm_mean_365D', 'trainer_place_mean_365D', 'train

100%|██████████| 25861/25861 [00:22<00:00, 1163.91it/s]


[(datetime.date(2018, 1, 2), datetime.date(2019, 3, 28)), (datetime.date(2019, 3, 28), datetime.date(2020, 6, 20)), (datetime.date(2020, 6, 20), datetime.date(2021, 9, 13)), (datetime.date(2021, 9, 13), datetime.date(2021, 12, 31))]
2019-03-28
2020-06-20
2021-09-13
2021-12-31
Train examples [15659, 15012, 14843, 3956]


100%|██████████| 25860/25860 [00:10<00:00, 2517.47it/s]
4it [02:09, 32.42s/it]


...

Quickpass time 12.485996800009161


 26%|██▌       | 655/2500 [12:12:58<32:10:39, 62.79s/it]

Quickpass time 17.89211200000136


 26%|██▋       | 657/2500 [12:15:20<33:59:10, 66.39s/it]

: 

In [ ]:
wandb_config_static.update({'learning_rate':0.001})
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static.update({'learning_rate':0.0005})
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static.update({'learning_rate':0.00008})
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static.update({'learning_rate':0.0001})
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static.update({'learning_rate':0.005})
model_pipeline(raceDB,config=wandb_config_static,sweep=False)